In [1]:
import os
import numpy as np
import nibabel as nib
from nibabel.cifti2 import Cifti2Image, cifti2_axes
import pandas as pd

In [2]:
dtseries = nib.load(
    '/scratch/jjeyachandra/multiband-censoring-comparisons/output/sub-LA1LA10032_ses-01_task-rest_run-1_space-MNI152NLin2009cAsym_desc-powers_cleaned.dtseries.nii'
)


dlabel = nib.load(
    '../../data/atlases/schaefer_2018/Schaefer2018_200Parcels_17Networks_order.dlabel.nii'
)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [3]:
ax_brain_ser = dtseries.header.get_axis(1)

In [4]:
ax_brain_ser.volume_shape

(91, 109, 91)

In [5]:
ax_label = dlabel.header.get_axis(0)
ax_brain_label = dlabel.header.get_axis(1)

Step 1: Match the dtseries/dlabel brain structures

In [6]:
# We need to construct our matrix first which will collapse
# the time-series axis and brain model axis into P x P

In [7]:
# Match brain models
matched_structs = [
    (dstruct, dslice, tslice) for dstruct, dslice, _
    in ax_brain_label.iter_structures()
    for tstruct, tslice, _
    in ax_brain_ser.iter_structures()
    if dstruct == tstruct
]

Using slices is incorrect. Need to use vertex mapping!

In [8]:
struct, dsl, tsl = matched_structs[0]
vtsl = ax_brain_ser.vertex[tsl]
print(struct, dsl, tsl)
vtsl.shape

CIFTI_STRUCTURE_CORTEX_LEFT slice(0, 32492, None) slice(0, 29696, None)


(29696,)

In [9]:
# Using tsl, pull information from dlabel
ser = dtseries.get_fdata()
lbl = dlabel.get_fdata().astype(int)

In [10]:
# Perform argsort
lbl_order = np.argsort(lbl[:,vtsl])[0]
lbl_slice = lbl[:,vtsl][:,lbl_order]
ser_slice = ser[:,tsl][:,lbl_order]

In [11]:
# Get index groupings and split
lbl_ids, lbl_inds = np.unique(lbl_slice, return_index=True)
arrs = np.split(ser_slice.T, lbl_inds[1:])

In [12]:
# Write into meants
zts = np.empty((lbl_ids.shape[0], dtseries.shape[0]))

# Write standardized ts
for i, a in enumerate(arrs):
    meants = a.mean(axis=0)
    zts[i] =(meants - meants.mean()) / meants.std()

In [13]:
# Compute pearson correlation matrix
R = (zts @ zts.T)
R /= R[0,0]

In [14]:
ax_label.label

array([{0: ('???', (1.0, 1.0, 1.0, 0.0)), 1: ('17Networks_LH_VisCent_ExStr_1', (0.470588, 0.0705882, 0.52549, 1.0)), 2: ('17Networks_LH_VisCent_ExStr_2', (0.470588, 0.0705882, 0.533333, 1.0)), 3: ('17Networks_LH_VisCent_Striate_1', (0.470588, 0.0705882, 0.537255, 1.0)), 4: ('17Networks_LH_VisCent_ExStr_3', (0.470588, 0.0705882, 0.541176, 1.0)), 5: ('17Networks_LH_VisCent_ExStr_4', (0.470588, 0.0705882, 0.545098, 1.0)), 6: ('17Networks_LH_VisCent_ExStr_5', (0.470588, 0.0705882, 0.54902, 1.0)), 7: ('17Networks_LH_VisPeri_ExStrInf_1', (1.0, 0.00392157, 0.00392157, 1.0)), 8: ('17Networks_LH_VisPeri_ExStrInf_2', (1.0, 0.00392157, 0.00784314, 1.0)), 9: ('17Networks_LH_VisPeri_ExStrInf_3', (1.0, 0.00392157, 0.0117647, 1.0)), 10: ('17Networks_LH_VisPeri_StriCal_1', (1.0, 0.00392157, 0.0156863, 1.0)), 11: ('17Networks_LH_VisPeri_ExStrSup_1', (1.0, 0.00392157, 0.0196078, 1.0)), 12: ('17Networks_LH_VisPeri_ExStrSup_2', (1.0, 0.00392157, 0.0235294, 1.0)), 13: ('17Networks_LH_SomMotA_1', (0.27451, 

In [259]:
# Now get column names
col_names = [
    ax_label.label[0][i][0] for i in lbl_ids
]

In [275]:
# Construct dataframe
df = pd.DataFrame(R, columns=col_names,
                 index=col_names)


In [276]:
# Pull upper-triangular
df = df.where(np.triu(np.ones(df.shape)).astype(np.bool))
df = df.stack().reset_index()
df.columns = ['Row','Column','Value']

<ipython-input-276-e19b67e5df71>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df = df.where(np.triu(np.ones(df.shape)).astype(np.bool))


In [277]:
df

,Row,Column,Value
0,???,???,1.000000
1,???,17Networks_LH_VisCent_ExStr_1,-0.141900
2,???,17Networks_LH_VisCent_ExStr_2,-0.063859
3,???,17Networks_LH_VisCent_Striate_1,-0.061757
4,???,17Networks_LH_VisCent_ExStr_3,-0.171102
...,...,...,...
5146,17Networks_LH_DefaultC_PHC_1,17Networks_LH_TempPar_1,0.031803
5147,17Networks_LH_DefaultC_PHC_1,17Networks_LH_TempPar_2,0.088496
5148,17Networks_LH_TempPar_1,17Networks_LH_TempPar_1,1.000000
5149,17Networks_LH_TempPar_1,17Networks_LH_TempPar_2,0.636917


This is within hemisphere but generalizing to both hemispheres and capturing inter-hemisphere connectivity is trivial